In [1]:
from typing import Counter
from core import *
from pydantic import ValidationError
from pprint import pprint

repo = github_client.get_repo("locustio/locust")

import difflib

stats = Counter()
for i, pr in enumerate(repo.get_pulls(state='open')):
    if i > 10:
        print("Stopping after 10 PRs")
        break

    try:
        result = review_pull_request(repo, pr)
        print(f"PR #{pr.number}: {result}")

        if result.should_update:
            stats["should_update"] += 1

            print(f"""

                  
CHANGES FOR PR #{pr.number}:

Reason:
{result.reason}

Updated README (diff):
""")
            
            diff = difflib.unified_diff(
                repo.get_readme().decoded_content.decode("utf-8").splitlines(),
                result.updated_readme.splitlines(),
            )
            for line in diff:
                print(line)

        else:
            stats["no_update"] += 1
    except ValidationError as e:
        stats["ValidationError"] += 1
        pprint(e.json())
    except ValueError as e:
        stats["ValueError"] += 1
        pprint(e)

pprint(stats)

Validation error, trying again
PR #2899: should_update=True reason='The README could benefit from some updates to improve its structure, readability, and content based on the provided README guidelines. The current README is good but can be enhanced to better highlight key features, installation process, and community engagement.' updated_readme='# Locust\n\n[![PyPI](https://img.shields.io/pypi/v/locust.svg)](https://pypi.org/project/locust/) [![Downloads](https://pepy.tech/badge/locust/week)](https://pepy.tech/project/locust) [![Build Status](https://github.com/locustio/locust/workflows/Tests/badge.svg)](https://github.com/locustio/locust/actions?query=workflow%3ATests) [![GitHub contributors](https://img.shields.io/github/contributors/locustio/locust.svg)](https://github.com/locustio/locust/graphs/contributors) [![Support Ukraine Badge](https://bit.ly/support-ukraine-now)](https://github.com/support-ukraine/support-ukraine)\n\n## What is Locust?\n\nLocust is an open-source performanc

In [2]:
# Dev log

## Before prompt engineering, running on Locust
Counter({'ValidationError': 3, 'should_update': 1})

## Stronger guidance in the prompt itself, like the Pydantic field descriptions and how they're mentioned in the prompt itself
Counter({'ValidationError': 4})
Counter({'ValidationError': 4})
Counter({'ValidationError': 2, 'should_update': 1, 'no_update': 1})

## Retries
Counter({'ValidationError': 3, 'should_update': 1})

## Prompt updates, Pydantic model updates
Counter({'should_update': 3, 'ValueError': 1})



Counter({'ValidationError': 3, 'should_update': 1})